# 🐱 Ideal Match Predictor (LangChain Edition)

This notebook uses **LangChain** to manage interactions with an LLM (like Gemma on Ollama). The primary goal is to create a more robust experience by ensuring the LLM does not repeat scenarios.

**Key Features:**
- **LangChain Prompt Templating**: Structures the requests to the LLM in a clean, reusable way.
- **History/Memory Management**: Tracks which scenarios have been used and instructs the LLM to generate unique new ones.

In [33]:
# 1) Setup & Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import json
import random
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import time

# --- LangChain Imports ---
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import Dict, List

In [34]:
# 2) Load Data and Define Schema
DATA_PATH = "./data.csv"
try:
    df = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print(f"Error: {DATA_PATH} not found. Make sure 'data.csv' is in the same directory.")
    df = pd.DataFrame({'name': ['Dummy'], 'summary': ['Dummy summary'], 'Trait1': [50], 'Trait2': [50]})

NAME_COL = "name"
SUMMARY_COL = "summary"
trait_cols = df.select_dtypes(include=np.number).columns.tolist()

print("Traits loaded:", trait_cols)

Traits loaded: ['Agency', 'Resilience', 'Nurturance', 'Assertiveness', 'Intellect', 'Combat Prowess', 'Emotional Stability', 'Loyalty', 'Cynicism', 'Optimism', 'Perseverance', 'Adaptability', 'Self-Esteem', 'Empathy', 'Impulsiveness', 'Discipline', 'Ambition', 'Social Acuity', 'Independence', 'Altruism']


In [35]:
# 3) Normalize and Center Traits
scaler = MinMaxScaler()
traits_norm = pd.DataFrame(scaler.fit_transform(df[trait_cols]) - 0.5, columns=trait_cols, index=df.index)
print("Traits normalized and centered.")

Traits normalized and centered.


### 4) LangChain-Powered LLM Interaction

Here, we define the core logic for interacting with the LLM using LangChain. We create a sophisticated prompt template that accepts a list of traits and the history of previously asked questions. This prevents repetition and ensures a varied experience.

In [36]:
# Updated shorter, more concise scenarios
ALL_SIMULATED_SCENARIOS = [
    {
      "scenario_question": "Festival day in your city. Your partner prefers to:",
      "option_a": "A) Mingle with crowds and meet new people",
      "option_b": "B) Find a quiet spot to watch fireworks together",
      "vector_a": { "Social Acuity": 2.0, "Optimism": 1.0 },
      "vector_b": { "Independence": 1.5, "Emotional Stability": 1.0 }
    },
    {
      "scenario_question": "A friend is wrongly accused of a crime. Your partner:",
      "option_a": "A) Immediately supports them with unwavering loyalty",
      "option_b": "B) Suggests examining evidence objectively first",
      "vector_a": { "Loyalty": 2.0, "Empathy": 1.5 },
      "vector_b": { "Cynicism": 1.5, "Intellect": 1.0 }
    },
    {
      "scenario_question": "Sudden magical threat appears. Your partner:",
      "option_a": "A) Acts on instinct, unleashing full power immediately",
      "option_b": "B) Takes defensive stance, analyzes weaknesses first",
      "vector_a": { "Combat Prowess": 2.0, "Impulsiveness": 1.5 },
      "vector_b": { "Discipline": 2.0, "Intellect": 1.5 }
    },
    {
        "scenario_question": "Ancient library discovered. Your partner wants to:",
        "option_a": "A) Dive into forbidden sections for powerful knowledge",
        "option_b": "B) Systematically catalog everything for preservation",
        "vector_a": { "Ambition": 2.0, "Impulsiveness": 1.0 },
        "vector_b": { "Discipline": 2.0, "Altruism": 1.5 }
    },
    {
        "scenario_question": "After a heartbreaking setback, your partner:",
        "option_a": "A) Shares feelings openly, seeking comfort",
        "option_b": "B) Retreats privately to process grief alone",
        "vector_a": { "Empathy": 1.0, "Nurturance": 1.0 },
        "vector_b": { "Resilience": 2.0, "Independence": 1.5 }
    },
    {
        "scenario_question": "You discover their secret talent. They:",
        "option_a": "A) Light up and eagerly share everything about it",
        "option_b": "B) Blush and downplay it as unimpressive",
        "vector_a": { "Self-Esteem": 2.0, "Optimism": 1.5 },
        "vector_b": { "Self-Esteem": -1.5, "Empathy": 1.0 }
    },
    {
        "scenario_question": "Group project disagreement. Your partner:",
        "option_a": "A) Takes charge with a clear plan",
        "option_b": "B) Listens to all sides, suggests compromise",
        "vector_a": { "Assertiveness": 2.0, "Ambition": 1.5 },
        "vector_b": { "Empathy": 2.0, "Adaptability": 1.5 }
    },
    {
        "scenario_question": "Dangerous mission opportunity arises. Your partner:",
        "option_a": "A) Volunteers immediately for the challenge",
        "option_b": "B) Carefully weighs risks before deciding",
        "vector_a": { "Combat Prowess": 1.5, "Impulsiveness": 2.0 },
        "vector_b": { "Intellect": 2.0, "Emotional Stability": 1.5 }
    },
    {
        "scenario_question": "Someone needs help but can't pay. Your partner:",
        "option_a": "A) Helps immediately without expecting anything",
        "option_b": "B) Explains they need to prioritize paying clients",
        "vector_a": { "Altruism": 2.0, "Nurturance": 1.5 },
        "vector_b": { "Cynicism": 1.5, "Discipline": 1.0 }
    },
    {
        "scenario_question": "Training session with others. Your partner:",
        "option_a": "A) Pushes themselves to outperform everyone",
        "option_b": "B) Focuses on helping weaker teammates improve",
        "vector_a": { "Ambition": 2.0, "Assertiveness": 1.5 },
        "vector_b": { "Nurturance": 2.0, "Empathy": 1.5 }
    },
    {
        "scenario_question": "Plans fall through last minute. Your partner:",
        "option_a": "A) Quickly improvises something fun instead",
        "option_b": "B) Gets frustrated and needs time to adjust",
        "vector_a": { "Adaptability": 2.0, "Optimism": 1.5 },
        "vector_b": { "Emotional Stability": -1.0, "Perseverance": 1.0 }
    },
    {
        "scenario_question": "Facing a moral dilemma. Your partner:",
        "option_a": "A) Does what feels right in their heart",
        "option_b": "B) Analyzes consequences logically first",
        "vector_a": { "Empathy": 2.0, "Impulsiveness": 1.0 },
        "vector_b": { "Intellect": 2.0, "Discipline": 1.5 }
    }
]

In [37]:
# ⚡ SPEED MODE TOGGLE ⚡
# Set to True for instant scenarios, False for LLM generation
SPEED_MODE = True  # Change to False if you want to wait for LLM

if SPEED_MODE:
    print("⚡ SPEED MODE ENABLED - Using instant pre-made scenarios")
    llm = None  # Force simulation mode
else:
    print("🐌 LLM MODE - Will generate unique scenarios (slower)")

⚡ SPEED MODE ENABLED - Using instant pre-made scenarios


In [38]:
class ScenarioGUI:
    def __init__(self, scenario_data):
        self.result = None
        self.root = tk.Tk()
        self.root.title("🎭 Ideal Match Predictor")
        self.root.geometry("800x500")
        self.root.configure(bg='#f0f0f0')
        
        # Make window center on screen
        self.root.eval('tk::PlaceWindow . center')
        
        # Main frame
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Title
        title_label = ttk.Label(main_frame, text="🎭 Choose Your Preference", 
                               font=('Arial', 16, 'bold'))
        title_label.pack(pady=(0, 20))
        
        # Scenario question
        scenario_frame = ttk.LabelFrame(main_frame, text="Scenario", padding="15")
        scenario_frame.pack(fill=tk.X, pady=(0, 20))
        
        scenario_text = ttk.Label(scenario_frame, text=scenario_data['scenario_question'], 
                                 font=('Arial', 12, 'bold'), wraplength=700)
        scenario_text.pack()
        
        # Options frame
        options_frame = ttk.LabelFrame(main_frame, text="Choose Your Option", padding="15")
        options_frame.pack(fill=tk.X, pady=(0, 20))
        
        # Option A
        option_a_label = ttk.Label(options_frame, text=scenario_data['option_a'], 
                                  font=('Arial', 11), wraplength=700, foreground='#0066cc')
        option_a_label.pack(anchor=tk.W, pady=(0, 10))
        
        # Option B  
        option_b_label = ttk.Label(options_frame, text=scenario_data['option_b'], 
                                  font=('Arial', 11), wraplength=700, foreground='#cc6600')
        option_b_label.pack(anchor=tk.W)
        
        # Buttons frame
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(pady=20)
        
        # Create buttons with better styling
        button_style = {'width': 12, 'font': ('Arial', 10, 'bold')}
        
        sa_btn = tk.Button(buttons_frame, text="Strongly A", bg='#0052cc', fg='white',
                          command=lambda: self.set_result(1.0, "a"), **button_style)
        sa_btn.pack(side=tk.LEFT, padx=5)
        
        a_btn = tk.Button(buttons_frame, text="Prefer A", bg='#66a3ff', fg='white',
                         command=lambda: self.set_result(0.5, "a"), **button_style)
        a_btn.pack(side=tk.LEFT, padx=5)
        
        n_btn = tk.Button(buttons_frame, text="Neutral", bg='#808080', fg='white',
                         command=lambda: self.set_result(0.0, "a"), **button_style)
        n_btn.pack(side=tk.LEFT, padx=5)
        
        b_btn = tk.Button(buttons_frame, text="Prefer B", bg='#ff9933', fg='white',
                         command=lambda: self.set_result(0.5, "b"), **button_style)
        b_btn.pack(side=tk.LEFT, padx=5)
        
        sb_btn = tk.Button(buttons_frame, text="Strongly B", bg='#cc4400', fg='white',
                          command=lambda: self.set_result(1.0, "b"), **button_style)
        sb_btn.pack(side=tk.LEFT, padx=5)
        
        # Keyboard shortcuts
        self.root.bind('1', lambda e: self.set_result(1.0, "a"))
        self.root.bind('2', lambda e: self.set_result(0.5, "a"))
        self.root.bind('3', lambda e: self.set_result(0.0, "a"))
        self.root.bind('4', lambda e: self.set_result(0.5, "b"))
        self.root.bind('5', lambda e: self.set_result(1.0, "b"))
        
        # Instructions
        instructions = ttk.Label(main_frame, 
                                text="💡 Click buttons or use keys: 1=Strongly A, 2=Prefer A, 3=Neutral, 4=Prefer B, 5=Strongly B",
                                font=('Arial', 9), foreground='#666666')
        instructions.pack(pady=(10, 0))
        
        # Make the window modal
        self.root.focus_set()
        self.root.grab_set()
        
    def set_result(self, multiplier, choice):
        self.result = (multiplier, choice)
        self.root.destroy()
        
    def get_choice(self):
        self.root.mainloop()
        return self.result

def ask_scenario_question(scenario_data):
    """Presents the scenario using a GUI and returns the user's preference."""
    print(f"\n🎭 Scenario: {scenario_data['scenario_question']}")
    print(f"  {scenario_data['option_a']}")
    print(f"  {scenario_data['option_b']}")
    print("📱 Opening GUI for your choice...")
    
    # Create and show GUI
    gui = ScenarioGUI(scenario_data)
    result = gui.get_choice()
    
    if result is None:
        print("❌ No choice made, defaulting to neutral")
        return (0.0, "a")
    
    multiplier, choice = result
    choice_text = {
        (1.0, "a"): "Strongly prefer A",
        (0.5, "a"): "Prefer A", 
        (0.0, "a"): "Neutral",
        (0.5, "b"): "Prefer B",
        (1.0, "b"): "Strongly prefer B"
    }
    
    print(f"✅ You chose: {choice_text.get((multiplier, choice), 'Unknown')}")
    return result

In [39]:
# Test the GUI scenario system
print("🧪 Testing GUI scenario generation...")
test_scenario = get_llm_scenario_with_langchain([])
print(f"📋 Generated scenario: {test_scenario['scenario_question']}")
print(f"🅰️  Option A: {test_scenario['option_a']}")
print(f"🅱️  Option B: {test_scenario['option_b']}")
print("✅ Scenario generation test successful!")
print("\n💡 Ready for GUI testing! Run the main loop to see the interactive interface.")

🧪 Testing GUI scenario generation...
🎭 Using simulation mode (LLM not available)
📋 Generated scenario: Festival day in your city. Your partner prefers to:
🅰️  Option A: A) Mingle with crowds and meet new people
🅱️  Option B: B) Find a quiet spot to watch fireworks together
✅ Scenario generation test successful!

💡 Ready for GUI testing! Run the main loop to see the interactive interface.


In [40]:
# 🎮 GUI Demo - Try the Interface!
print("🎮 GUI DEMO - Test the button interface!")
print("💡 To test the GUI, uncomment and run the line below:")
print("   demo_result = demo_gui()")
print()

def demo_gui():
    demo_scenario = {
        "scenario_question": "Demo: How do you prefer to spend your weekend?",
        "option_a": "A) Go out and explore new places with friends",
        "option_b": "B) Stay home and enjoy a quiet, relaxing day",
        "vector_a": {"Social Acuity": 2.0, "Optimism": 1.0},
        "vector_b": {"Independence": 1.5, "Emotional Stability": 1.0}
    }
    
    print("🎭 Opening demo GUI window...")
    result = ask_scenario_question(demo_scenario)
    print(f"🎉 Demo complete! You chose: {result}")
    return result

print("✅ Demo function ready! Uncomment the line below to test:")
print("# demo_result = demo_gui()")
print()
print("🎯 Or skip the demo and run the main assessment in cell 13!")

🎮 GUI DEMO - Test the button interface!
💡 To test the GUI, uncomment and run the line below:
   demo_result = demo_gui()

✅ Demo function ready! Uncomment the line below to test:
# demo_result = demo_gui()

🎯 Or skip the demo and run the main assessment in cell 13!


In [41]:
# Comprehensive LLM Test - Generate multiple unique scenarios
print("🚀 Comprehensive LLM Test - Generating multiple scenarios...")
print("=" * 60)

test_history = []
for i in range(3):
    print(f"\n🎭 Generating scenario {i+1}...")
    scenario = get_llm_scenario_with_langchain(test_history)
    test_history.append(scenario['scenario_question'])
    
    print(f"📝 Scenario: {scenario['scenario_question']}")
    print(f"🅰️  {scenario['option_a']}")
    print(f"🅱️  {scenario['option_b']}")
    print(f"📊 Vector A traits: {list(scenario['vector_a'].keys())}")
    print(f"📊 Vector B traits: {list(scenario['vector_b'].keys())}")
    print("-" * 40)

print("✅ LLM integration is working perfectly!")
print("🎉 The notebook is ready for interactive use!")

🚀 Comprehensive LLM Test - Generating multiple scenarios...

🎭 Generating scenario 1...
🎭 Using simulation mode (LLM not available)
📝 Scenario: Festival day in your city. Your partner prefers to:
🅰️  A) Mingle with crowds and meet new people
🅱️  B) Find a quiet spot to watch fireworks together
📊 Vector A traits: ['Social Acuity', 'Optimism']
📊 Vector B traits: ['Independence', 'Emotional Stability']
----------------------------------------

🎭 Generating scenario 2...
🎭 Using simulation mode (LLM not available)
📝 Scenario: A friend is wrongly accused of a crime. Your partner:
🅰️  A) Immediately supports them with unwavering loyalty
🅱️  B) Suggests examining evidence objectively first
📊 Vector A traits: ['Loyalty', 'Empathy']
📊 Vector B traits: ['Cynicism', 'Intellect']
----------------------------------------

🎭 Generating scenario 3...
🎭 Using simulation mode (LLM not available)
📝 Scenario: Sudden magical threat appears. Your partner:
🅰️  A) Acts on instinct, unleashing full power imme

## ✅ GUI Integration Complete - Interactive & Fast!

The notebook now features a beautiful Tkinter GUI interface for easy interaction:

### 🎮 **GUI Features:**
- **Interactive Buttons**: Click SA, A, N, B, SB for your preferences
- **Keyboard Shortcuts**: Use keys 1-5 for lightning-fast responses  
- **Visual Design**: Clean, modern interface with color-coded options
- **Modal Windows**: Focused experience with each scenario
- **Progress Tracking**: See your choices and profile updates in real-time

### ⚡ **Speed Optimizations:**
- **Speed Mode Toggle**: Instant scenarios vs LLM generation
- **Faster Models**: Prioritizes `llama3.2:1b` (1B params) over larger models
- **Shorter Scenarios**: Concise, focused scenarios instead of verbose text
- **Reduced Questions**: 7 scenarios instead of 12 for faster completion
- **One-Click Choices**: No typing required - just click buttons!

### 🎯 **How to Use:**
1. **Run cell 13** (Main Profiling Loop) to start the assessment
2. **GUI windows appear** for each scenario - click your preference
3. **Choose from 5 options**: Strongly A → Prefer A → Neutral → Prefer B → Strongly B
4. **Use keyboard shortcuts** (1-5) for even faster responses
5. **See your results** with beautiful formatting and match percentages

### 🚀 **Quick Start:**
- **Demo the GUI**: Uncomment and run the demo in cell 10
- **Full Assessment**: Run cell 13 for complete interactive experience
- **Speed Mode**: Currently enabled for instant scenarios
- **LLM Mode**: Set `SPEED_MODE = False` for AI-generated content

**Ready to find your ideal match with style! 🎉**

### 5) Main Profiling Loop

The main loop now initializes and maintains a `scenario_history` list, passing it to the LangChain-powered function in each iteration. This ensures you get a new scenario every time.

In [ ]:
user_preferences = pd.Series(0.0, index=trait_cols)
scenario_history = []
MAX_QUESTIONS = 7  # Reduced from 12 for faster completion

print("🎉 Starting Interactive Ideal Match Assessment with GUI!")
print("=" * 60)
print("💡 A GUI window will appear for each scenario - click your preference!")
print("⌨️  Or use keyboard shortcuts: 1-5 for different preference levels")
print("=" * 60)

for q_num in range(1, MAX_QUESTIONS + 1):
    print(f"\n--- Scenario {q_num}/{MAX_QUESTIONS} ---")
    
    # Get a new, unique scenario using our function
    scenario = get_llm_scenario_with_langchain(history=scenario_history)
    
    # Add the new question to our history to prevent repeats
    scenario_history.append(scenario['scenario_question'])
    
    # Show GUI and get user choice
    multiplier, choice = ask_scenario_question(scenario)
    
    # Update preferences based on choice
    if multiplier > 0:
        chosen_vector_key = "vector_a" if choice == "a" else "vector_b"
        score_vector = scenario[chosen_vector_key]
        print(f"  📊 Your choice updated your profile by: {score_vector}")
        for trait, score in score_vector.items():
            if trait in user_preferences:
                user_preferences[trait] += score * multiplier
    else:
        print(f"  ➡️  Neutral choice - no profile changes")
    
    # Small delay for readability
    time.sleep(0.5)

print("\n" + "=" * 60)
print("🎯 PROFILE COMPLETE!")
print("=" * 60)
print("📈 Your personality trait preferences:")
significant_preferences = user_preferences[abs(user_preferences) > 0.1].sort_values(ascending=False)
if len(significant_preferences) > 0:
    for trait, score in significant_preferences.items():
        direction = "↗️" if score > 0 else "↘️"
        print(f"  {direction} {trait}: {score:.2f}")
else:
    print("  🤷 You have a very balanced profile with no strong preferences!")
print("=" * 60)

🎉 Starting Interactive Ideal Match Assessment with GUI!
💡 A GUI window will appear for each scenario - click your preference!
⌨️  Or use keyboard shortcuts: 1-5 for different preference levels

--- Scenario 1/7 ---
🎭 Using simulation mode (LLM not available)

🎭 Scenario: Festival day in your city. Your partner prefers to:
  A) Mingle with crowds and meet new people
  B) Find a quiet spot to watch fireworks together
📱 Opening GUI for your choice...
✅ You chose: Strongly prefer A
  📊 Your choice updated your profile by: {'Social Acuity': 2.0, 'Optimism': 1.0}
✅ You chose: Strongly prefer A
  📊 Your choice updated your profile by: {'Social Acuity': 2.0, 'Optimism': 1.0}

--- Scenario 2/7 ---
🎭 Using simulation mode (LLM not available)

🎭 Scenario: A friend is wrongly accused of a crime. Your partner:
  A) Immediately supports them with unwavering loyalty
  B) Suggests examining evidence objectively first
📱 Opening GUI for your choice...

--- Scenario 2/7 ---
🎭 Using simulation mode (LLM n

### 6) Final Ranking

The final ranking logic remains the same. We calculate the dot product of your final preference vector against every character's trait vector to find the best match.

In [ ]:
match_scores = traits_norm.values @ user_preferences.values
rank_df = pd.DataFrame({
    NAME_COL: df[NAME_COL],
    SUMMARY_COL: df[SUMMARY_COL],
    "match_score": match_scores
})
rank_df = rank_df.sort_values("match_score", ascending=False).reset_index(drop=True)

print("\n" + "=" * 70)
print("🏆 YOUR TOP IDEAL MATCHES! 🏆")
print("=" * 70)

top_matches = rank_df.head(5)
for i, row in top_matches.iterrows():
    rank_emoji = ["🥇", "🥈", "🥉", "🏅", "🏅"][i]
    match_percentage = max(0, min(100, (row['match_score'] + 0.5) * 100))
    
    print(f"\n{rank_emoji} #{i+1}: {row[NAME_COL]}")
    print(f"   💖 Match Score: {match_percentage:.1f}%")
    print(f"   📖 {row[SUMMARY_COL]}")
    print("-" * 50)

print(f"\n🎯 Analysis complete! Your ideal match profile favors:")
if len(significant_preferences) > 0:
    top_traits = significant_preferences.head(3)
    for trait, score in top_traits.items():
        print(f"   ✨ {trait} ({score:+.2f})")
else:
    print("   🌟 A balanced personality - you're open to many types!")

print(f"\n💡 Based on {MAX_QUESTIONS} scenarios, your top match is: {rank_df.iloc[0][NAME_COL]}!")
print("=" * 70)

# 📊 DETAILED STATISTICS & RAW DATA
print("\n" + "=" * 80)
print("📊 DETAILED STATISTICS & RAW DATA")
print("=" * 80)

print("\n🔢 RAW PREFERENCE SCORES (All Traits):")
print("-" * 50)
for trait in trait_cols:
    score = user_preferences[trait]
    if abs(score) > 0.01:  # Show any non-zero preferences
        print(f"  {trait:<20}: {score:+6.3f}")
    else:
        print(f"  {trait:<20}:  0.000")

print(f"\n📈 STATISTICAL SUMMARY:")
print("-" * 50)
non_zero_prefs = user_preferences[user_preferences != 0]
print(f"  Total scenarios completed: {MAX_QUESTIONS}")
print(f"  Traits with preferences: {len(non_zero_prefs)}/{len(user_preferences)}")
print(f"  Strongest preference: {user_preferences.abs().max():.3f}")
print(f"  Average absolute preference: {user_preferences.abs().mean():.3f}")
print(f"  Preference range: {user_preferences.min():.3f} to {user_preferences.max():.3f}")

print(f"\n🎭 SCENARIO HISTORY:")
print("-" * 50)
for i, scenario in enumerate(scenario_history, 1):
    print(f"  {i}. {scenario}")

print(f"\n🏆 COMPLETE RANKING (All Characters):")
print("-" * 50)
for i, row in rank_df.iterrows():
    raw_score = row['match_score']
    match_percentage = max(0, min(100, (raw_score + 0.5) * 100))
    print(f"  {i+1:2d}. {row[NAME_COL]:<20} | Score: {raw_score:+6.3f} | Match: {match_percentage:5.1f}%")

print(f"\n🔍 TOP 5 DETAILED BREAKDOWN:")
print("-" * 50)
top_5 = rank_df.head(5)
for i, row in top_5.iterrows():
    char_name = row[NAME_COL]
    char_idx = df[df[NAME_COL] == char_name].index[0]
    char_traits = traits_norm.iloc[char_idx]
    
    print(f"\n#{i+1}: {char_name}")
    print(f"  Raw match score: {row['match_score']:+6.3f}")
    print(f"  Top character traits:")
    top_char_traits = char_traits.abs().nlargest(3)
    for trait in top_char_traits.index:
        char_val = char_traits[trait]
        your_pref = user_preferences[trait]
        compatibility = char_val * your_pref if your_pref != 0 else 0
        print(f"    {trait}: {char_val:+6.3f} (your pref: {your_pref:+6.3f}, compatibility: {compatibility:+6.3f})")

print("\n" + "=" * 80)
print("📋 ANALYSIS COMPLETE - Raw data available above for detailed review!")
print("=" * 80)